In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from time import sleep
import json
import datetime

In [3]:
# edit these three variables
user = 'realdonaldtrump'
start = datetime.datetime(2019, 8, 12)  # year, month, day
end = datetime.datetime(2019, 10, 3)  # year, month, day

# only edit these if you're having problems
delay = 1  # time to wait on each page load before reading the page
driver = webdriver.Safari()  # options are Chrome() Firefox() Safari()


# don't mess with this stuff
twitter_ids_filename = 'all_ids.json'
days = (end - start).days + 1
id_selector = '.time a.tweet-timestamp'
tweet_selector = 'li.js-stream-item'
user = user.lower()
ids = []

def format_day(date):
    day = '0' + str(date.day) if len(str(date.day)) == 1 else str(date.day)
    month = '0' + str(date.month) if len(str(date.month)) == 1 else str(date.month)
    year = str(date.year)
    return '-'.join([year, month, day])

def form_url(since, until):
    p1 = 'https://twitter.com/search?f=tweets&vertical=default&q=from%3A'
    p2 =  user + '%20since%3A' + since + '%20until%3A' + until + 'include%3Aretweets&src=typd'
    return p1 + p2

def increment_day(date, i):
    return date + datetime.timedelta(days=i)

for day in range(days):
    d1 = format_day(increment_day(start, 0))
    d2 = format_day(increment_day(start, 1))
    url = form_url(d1, d2)
    print(url)
    print(d1)
    driver.get(url)
    sleep(delay)

    try:
        found_tweets = driver.find_elements_by_css_selector(tweet_selector)
        increment = 10

        while len(found_tweets) >= increment:
            print('scrolling down to load more tweets')
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(delay)
            found_tweets = driver.find_elements_by_css_selector(tweet_selector)
            increment += 10

        print('{} tweets found, {} total'.format(len(found_tweets), len(ids)))

        for tweet in found_tweets:
            try:
                id = tweet.find_element_by_css_selector(id_selector).get_attribute('href').split('/')[-1]
                ids.append(id)
            except StaleElementReferenceException as e:
                print('lost element reference', tweet)

    except NoSuchElementException:
        print('no tweets on this day')

    start = increment_day(start, 1)


try:
    with open(twitter_ids_filename) as f:
        all_ids = ids + json.load(f)
        data_to_write = list(set(all_ids))
        print('tweets found on this scrape: ', len(ids))
        print('total tweet count: ', len(data_to_write))
except FileNotFoundError:
    with open(twitter_ids_filename, 'w') as f:
        all_ids = ids
        data_to_write = list(set(all_ids))
        print('tweets found on this scrape: ', len(ids))
        print('total tweet count: ', len(data_to_write))

with open(twitter_ids_filename, 'w') as outfile:
    json.dump(data_to_write, outfile)

print('all done here')
driver.close()

https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2019-08-12%20until%3A2019-08-13include%3Aretweets&src=typd
2019-08-12
4 tweets found, 0 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2019-08-13%20until%3A2019-08-14include%3Aretweets&src=typd
2019-08-13
scrolling down to load more tweets
17 tweets found, 4 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2019-08-14%20until%3A2019-08-15include%3Aretweets&src=typd
2019-08-14
scrolling down to load more tweets
17 tweets found, 21 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2019-08-15%20until%3A2019-08-16include%3Aretweets&src=typd
2019-08-15
scrolling down to load more tweets
16 tweets found, 38 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2019-08-16%20until%3A2019-08-17include%3Aretweets&src=typd
2019-08-16
scro

9 tweets found, 564 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2019-09-18%20until%3A2019-09-19include%3Aretweets&src=typd
2019-09-18
scrolling down to load more tweets
17 tweets found, 573 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2019-09-19%20until%3A2019-09-20include%3Aretweets&src=typd
2019-09-19
7 tweets found, 590 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2019-09-20%20until%3A2019-09-21include%3Aretweets&src=typd
2019-09-20
9 tweets found, 597 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2019-09-21%20until%3A2019-09-22include%3Aretweets&src=typd
2019-09-21
scrolling down to load more tweets
16 tweets found, 606 total
https://twitter.com/search?f=tweets&vertical=default&q=from%3Arealdonaldtrump%20since%3A2019-09-22%20until%3A2019-09-23include%3Aretweets&src=typd
2019-09-22
scrollin

In [ ]:
import tweepy
import json
import math
import glob
import csv
import zipfile
import zlib
from tweepy import TweepError
from time import sleep

# CHANGE THIS TO THE USER YOU WANT
user = 'realdonaldtrump'


with open('api_keys.json') as f:
    keys = json.load(f)

auth = tweepy.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth)
user = user.lower()
output_file = '{}.json'.format(user)
output_file_short = '{}_short.json'.format(user)
compression = zipfile.ZIP_DEFLATED

with open('all_ids.json') as f:
    ids = json.load(f)

print('total ids: {}'.format(len(ids)))

all_data = []
start = 0
end = 100
limit = len(ids)
i = math.ceil(limit / 100)

for go in range(i):
    print('currently getting {} - {}'.format(start, end))
    sleep(6)  # needed to prevent hitting API rate limit
    id_batch = ids[start:end]
    start += 100
    end += 100
    tweets = api.statuses_lookup(id_batch)
    for tweet in tweets:
        all_data.append(dict(tweet._json))

print('metadata collection complete')
print('creating master json file')
with open(output_file, 'w') as outfile:
    json.dump(all_data, outfile)

print('creating ziped master json file')
zf = zipfile.ZipFile('{}.zip'.format(user), mode='w')
zf.write(output_file, compress_type=compression)
zf.close()

results = []

def is_retweet(entry):
    return 'retweeted_status' in entry.keys()

def get_source(entry):
    if '<' in entry["source"]:
        return entry["source"].split('>')[1].split('<')[0]
    else:
        return entry["source"]

with open(output_file) as json_data:
    data = json.load(json_data)
    for entry in data:
        t = {
            "created_at": entry["created_at"],
            "text": entry["text"],
            "in_reply_to_screen_name": entry["in_reply_to_screen_name"],
            "retweet_count": entry["retweet_count"],
            "favorite_count": entry["favorite_count"],
            "source": get_source(entry),
            "id_str": entry["id_str"],
            "is_retweet": is_retweet(entry)
        }
        results.append(t)

print('creating minimized json master file')
with open(output_file_short, 'w') as outfile:
    json.dump(results, outfile)

with open(output_file_short) as master_file:
    data = json.load(master_file)
    fields = ["favorite_count", "source", "text", "in_reply_to_screen_name", "is_retweet", "created_at", "retweet_count", "id_str"]
    print('creating CSV version of minimized json master file')
    f = csv.writer(open('{}23-08-19_trump.csv'.format(user), 'w'))
    f.writerow(fields)
    for x in data:
        f.writerow([x["favorite_count"], x["source"], x["text"], x["in_reply_to_screen_name"], x["is_retweet"], x["created_at"], x["retweet_count"], x["id_str"]])

total ids: 18906
currently getting 0 - 100
currently getting 100 - 200
currently getting 200 - 300
currently getting 300 - 400
currently getting 400 - 500
currently getting 500 - 600
currently getting 600 - 700
currently getting 700 - 800
currently getting 800 - 900
currently getting 900 - 1000
currently getting 1000 - 1100
currently getting 1100 - 1200
currently getting 1200 - 1300
currently getting 1300 - 1400
currently getting 1400 - 1500
currently getting 1500 - 1600
currently getting 1600 - 1700
currently getting 1700 - 1800
currently getting 1800 - 1900
currently getting 1900 - 2000
currently getting 2000 - 2100
currently getting 2100 - 2200
currently getting 2200 - 2300
currently getting 2300 - 2400
currently getting 2400 - 2500
currently getting 2500 - 2600
currently getting 2600 - 2700
currently getting 2700 - 2800
currently getting 2800 - 2900
currently getting 2900 - 3000
currently getting 3000 - 3100
currently getting 3100 - 3200
currently getting 3200 - 3300
currently gett

In [41]:
import pandas as pd

In [42]:
df = pd.read_csv('realdonaldtrump.csv')

In [49]:
df.head()

,favorite_count,source,text,in_reply_to_screen_name,is_retweet,created_at,retweet_count,id_str
0,7318,Twitter for iPhone,"I will beat Hillary easily, but Lindsey Graham...",NaN,False,Wed Feb 17 14:40:24 +0000 2016,2826,699966633918525441
1,5469,Twitter for Android,Will go back on for a final question now!,NaN,False,Wed May 18 00:57:16 +0000 2016,970,732736781351849989
2,14033,Twitter for iPhone,Will be doing @oreillyfactor tonight at 8pm. E...,NaN,False,Thu Aug 11 22:01:30 +0000 2016,3552,763857902490836992
3,11183,Twitter for Android,"Ohio Gov.Kasich voted for NAFTA, from which Oh...",NaN,False,Sun Mar 13 11:58:17 +0000 2016,5243,708985532483837953
4,4504,Twitter for Android,"""@gerriweth: @megynkelly @realDonaldTrump Made...",NaN,False,Wed May 18 00:32:39 +0000 2016,923,732730584695463936


In [51]:
df.created_at = pd.to_datetime(df.created_at)

In [54]:
df.sort_values('created_at')

,favorite_count,source,text,in_reply_to_screen_name,is_retweet,created_at,retweet_count,id_str
519,5844,Twitter for Android,"I will be on @FoxNews live, with members of m...",NaN,False,2016-01-01 00:44:14+00:00,1825,682723973449289728
1264,7503,Twitter for iPhone,HAPPY NEW YEAR &amp; THANK YOU! https://t.co/Y...,NaN,False,2016-01-01 03:25:27+00:00,2961,682764544402440192
1458,7995,Twitter for iPhone,#HappyNewYearAmerica! https://t.co/EeQb8PDrUe,NaN,False,2016-01-01 05:18:23+00:00,2956,682792967736848385
315,7359,Twitter for iPhone,Happy New Year from #MarALago! Thank you to my...,NaN,False,2016-01-01 06:07:28+00:00,1719,682805320217980929
1587,6524,Twitter for Android,"""@jallenaip: Hillary said she was in a ""Fog of...",NaN,False,2016-01-01 06:08:06+00:00,2384,682805477168779264
3117,2903,Twitter for Android,"""@sprinklermanus: @CNN @realDonaldTrump they'r...",NaN,False,2016-01-01 21:29:56+00:00,792,683037464504745985
2967,7429,Twitter for Android,"Well, the year has officially begun. I have ma...",NaN,False,2016-01-01 23:00:09+00:00,2277,683060169677344768
137,2419,Twitter for Android,"""@marie7777777777: @realDonaldTrump we love u ...",NaN,False,2016-01-01 23:02:05+00:00,567,683060654098530305
1847,2693,Twitter for Android,"""@JodiL792: We are Standing with you! Spreadin...",NaN,False,2016-01-01 23:06:09+00:00,775,683061678246903808
2660,3057,Twitter for Android,"""@MEMEoryHead: I'm one of your biggest fans Mr...",NaN,False,2016-01-01 23:08:18+00:00,771,683062220490715136


In [ ]:
df.created_at.day